In [30]:
import Pkg
dep_installed = [val.name for val ∈ values(Pkg.dependencies())]
meta_array = ["Parameters" "PyPlot" "NPZ" "StatsBase" "Combinatorics" "IterTools" "JLD2" "JSON"]

for pkg_dep ∈ meta_array
    if pkg_dep ∉ dep_installed 
        Pkg.add(pkg_dep)
    end
end

In [1]:
using Distributed
using PyPlot
using NPZ
using JSON
using Statistics

include("./src/utils.jl")
include("./src/k_mc.jl")
include("./src/r_mc.jl")
include("./src/r_sqs.jl")
include("./src/k_sqs.jl")
include("./src/config.jl")

using .Utils
using .k_sqs 
using .r_sqs
using .k_mc
using .r_mc

println(Threads.nthreads())

24


Generate specified number of random sequences (e.g. Ising-like spin for each element specie, 2 for Cr, 1 for Mn, -1 for Co, -2 for Ni) under given composition. No need to generate random sequences when use SQS-based method.

In [ ]:
compo = [1/4, 1/4, 1/4, 1/4] #* specify your composition here: follow the Cr-Mn-Co-Ni order

compo_denote = vec2str(compo)
println("denotion for current composition: $(compo_denote)")

atom_num = 256 #* specify the number of atoms in one supercell (cell size)
gen_num = 10 #* specify the number of random sequences (supercell) to generate

println("number of generated random sequences: $(gen_num)")
println("number of atoms in lattice: $(atom_num)")
mat_ = ele_list_gen(compo, atom_num, gen_num, "int")

println("example for a random sequence: $(mat_[1,:])")

#* save generated sequences
pth_sav = "examples/mc/embed_sequence/mc_seq_$(compo_denote)_$(atom_num).npy"
println("Random sequences saved as $(pth_sav)")
npzwrite(pth_sav, mat_)

Based on the **existed random sequences** (can be generated by the cell above), derive r+k-space correlation function matrices, one can also use SQS-based sequences with specified randomness (not included in current program). The correlation functions are input variables that determine the MSAD as output for each input configuration (see next cellblock).

In [ ]:
atom_num = 256 #* number of atoms in the supercell
compo = [1/4, 1/4, 1/4, 1/4] #* specify your composition here: follow the Cr-Mn-Co-Ni order

@assert abs(sum(compo)-1) < 0.001 "Make sure the atomic contents sum up to 1"
compo_denote = vec2str(compo)

embed_pth = "examples/mc/embed_sequence/mc_seq_$(compo_denote)_$(atom_num).npy"

if !isfile(embed_pth)
    println("Embedding sequences not existed, check the composition and cell size")
end

embed_list = npzread(embed_pth)
println("Current composition: $(compo_denote) Number and Size of random sequences: $(size(embed_list))")

vec_mat_r_ = main_mc_r(1:size(embed_list, 1), atom_num, embed_list)
vec_mat_r = hcat(vec_mat_r_...)'
vec_mat_k_ = main_mc_k(atom_num, embed_list)
vec_mat = hcat(vec_mat_r, vec_mat_k_)

npzwrite(joinpath("examples/mc/correlate_func/vec_mat_$(compo_denote)_$(atom_num)_mc.npy"), vec_mat)
println("Correlation functions saved as examples/mc/correlate_func/vec_mat_$(compo_denote)_$(atom_num)_mc.npy")

Derive correlation function vectors based on **SQS**, one-to-one mapping for each composition-SQS pair. 

In [14]:
atom_num = 500 #* number of atoms in the supercell
compo_list = Matrix{Float64}([
    # 0.25 0.25 0.25 0.25; # CrMnCoNi
    # 1/3 0 1/3 1/3; # CrCoNi
    # 1/2 0 1/3 1/6; # nonequiatomic Cr-Mn-Co-Ni
    0.4 0.05 0.275 0.275
]) #* specify the composition (list) here: follow the Cr-Mn-Co-Ni order

cprvec_sqs_r = main_sqs_r(atom_num, compo_list)
cprvec_sqs_k = main_sqs_k(atom_num, compo_list)

for i in 1:size(compo_list, 1)
    compo_ = compo_list[i,:]
    compo_denote = vec2str(compo_)
    cprvec_sqs = vcat(cprvec_sqs_r[i], cprvec_sqs_k[i])
    cprvec_sqs = reshape(cprvec_sqs, 1, length(cprvec_sqs))

    pth_sav = "examples/sqs/fig/vec_mat_$(compo_denote)_$(atom_num)_sqs.npy"
    println("save as $(pth_sav)")
    npzwrite(pth_sav, cprvec_sqs)
end

save as examples/sqs/fig/vec_mat_4052828_500_sqs.npy


Evaluate the $MSAD^{1/2}$ using provided coefficients.

In [ ]:
coef_ = npzread("data/coef/coef_best_mix_108_192_32_240623.npy")
optim_sdf = npzread("data/coef/chosen_ind_mix_108_192_32_240623.npy")

compo = [1/4, 1/4, 1/4, 1/4] #* specify your composition here: follow the Cr-Mn-Co-Ni order
atom_num = 256 #* specify the number of atoms in the supercell
@assert abs(sum(compo)-1) < 0.001 "Make sure the atomic contents sum up to 1"
compo_denote = vec2str(compo)

#* option1: use atomic configuration-derived correlation functions as input
# vec_mat = npzread("examples/mc/correlate_func/vec_mat_$(compo_denote)_$(atom_num)_mc.npy")
# msad_mc = eval_mix(coef_, vec_mat[:,optim_sdf.+1])
# print("μ(MSAD) = $(mean(msad_mc)) σ(MSAD) = $(std(msad_mc))")

#* option2: use virtual SQS-derived correlation functions as input
vec_mat_sqs = npzread("examples/sqs/vec_mat_$(compo_denote)_$(atom_num)_sqs.npy")
msad_sqs = eval_mix(coef_, vec_mat_sqs[:,optim_sdf.+1])
println("SQS-based MSAD: $(msad_sqs)")